In [ ]:
!pip install skyfield

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz
from skyfield.api import load, wgs84
from skyfield.data import hipparcos

# -------------------------------------------------------------------
# CONFIGURATION
# -------------------------------------------------------------------
number_of_nights = 1        # <---- CHANGE THIS
images_per_night = 50       # <---- CHANGE THIS

# Observer location (Hanoi)
lat = 20.994838067341455
lon = 105.86773827462262
tz = pytz.timezone('Asia/Ho_Chi_Minh')

# Time window each night: 8 PM → 4 AM = 8 hours
window_hours = 8

# Start time: tonight at 20:00
start_date = datetime.now(tz).replace(hour=20, minute=0, second=0, microsecond=0)

# -------------------------------------------------------------------
# Generate timestamps
# -------------------------------------------------------------------
times = []

for day in range(number_of_nights):
    night_start = start_date + timedelta(days=day)
    night_end = night_start + timedelta(hours=window_hours)

    # Compute interval automatically
    total_minutes = window_hours * 60
    interval_minutes = total_minutes / (images_per_night - 1) if images_per_night > 1 else total_minutes

    for i in range(images_per_night):
        t = night_start + timedelta(minutes=i * interval_minutes)
        times.append(t)

print(f"\nGenerating {len(times)} images...")
print(f" Nights: {number_of_nights}")
print(f" Images per night: {images_per_night}")
print(f" Time interval per image: {interval_minutes:.2f} minutes")
print(f" First: {times[0].strftime('%Y-%m-%d %H:%M')}")
print(f" Last : {times[-1].strftime('%Y-%m-%d %H:%M')}")

# -------------------------------------------------------------------
# Load Skyfield data
# -------------------------------------------------------------------
ts = load.timescale()
planets = load('de421.bsp')
earth = planets['earth']
observer = earth + wgs84.latlon(lat, lon)

print("\nLoading Hipparcos star catalog...")
with load.open(hipparcos.URL) as f:
    df = hipparcos.load_dataframe(f)
print(f"Loaded {len(df)} stars")

# Filter bright stars
max_magnitude = 4.0
visible_df = df[df['magnitude'] <= max_magnitude].copy()
print(f"{len(visible_df)} stars remain (magnitude ≤ {max_magnitude})")

from skyfield.api import Star as SkyfieldStar
stars = SkyfieldStar.from_dataframe(visible_df)
magnitudes = visible_df['magnitude'].values

# -------------------------------------------------------------------
# Projection
# -------------------------------------------------------------------
def stereo_project(alt, az):
    r = 2 * np.tan(np.radians(90 - alt) / 2)
    x = r * np.sin(np.radians(az))
    y = r * np.cos(np.radians(az))
    return x, y

# -------------------------------------------------------------------
# Image generation
# -------------------------------------------------------------------
print("\nGenerating sky plots...\n")

for idx, obs_time in enumerate(times):
    night = idx // images_per_night + 1
    img_idx = idx % images_per_night + 1

    print(f"  [Night {night}/{number_of_nights}, Image {img_idx}/{images_per_night}] "
          f"{obs_time.strftime('%Y-%m-%d %H:%M')}...", end="")

    t = ts.from_datetime(obs_time)

    # Observe stars
    astrometric = observer.at(t).observe(stars)
    alt, az, distance = astrometric.apparent().altaz()

    alt_deg = alt.degrees
    az_deg = az.degrees

    # Only stars above horizon
    mask = alt_deg > 0
    alt_vis = alt_deg[mask]
    az_vis = az_deg[mask]
    mag_vis = magnitudes[mask]

    # Project
    x, y = stereo_project(alt_vis, az_vis)

    # Star sizes
    sizes = 100 * np.exp(-mag_vis / 2.0)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 12), facecolor='white')
    ax.set_facecolor('white')

    ax.scatter(x, y, s=sizes, c='black', alpha=0.85, linewidths=0, edgecolors='none')

    # Horizon circle
    circle = plt.Circle((0, 0), 2, fill=False, color='black', linewidth=0.5, alpha=0.25)
    ax.add_patch(circle)

    # Directions
    props = dict(fontsize=11, color='black', alpha=0.4)
    ax.text(0, 2.15, 'N', ha='center', **props)
    ax.text(0, -2.15, 'S', ha='center', **props)
    ax.text(2.15, 0, 'E', ha='center', **props)
    ax.text(-2.15, 0, 'W', ha='center', **props)

    ax.set_aspect('equal')
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.axis('off')

    # Title
    info = obs_time.strftime('%Y-%m-%d %H:%M ICT')
    ax.text(0, -2.8,
            f'Hanoi Night Sky (≤ {max_magnitude} mag)\n'
            f'{info}\n{lat:.2f}°N, {lon:.2f}°E',
            ha='center', va='top', fontsize=9, color='black', alpha=0.45)

    plt.tight_layout()

    # Filename
    filename = f"hanoi_sky_night{night:02d}_{obs_time.strftime('%Y%m%d_%H%M')}.png"
    plt.savefig(filename, dpi=150, facecolor='white', bbox_inches='tight')
    plt.close()

    print(" saved")

print("\nComplete!")
print(f" Total images: {len(times)}")
print(" Files saved as: hanoi_sky_nightNN_YYYYMMDD_HHMM.png")


In [ ]:
!zip -r /content/hanoi_sky.zip /content/hanoi_sky_night*